In [1]:
import numpy as np 
import pandas as pd
import pickle

In [2]:
# import river modules
from river import datasets
from river import evaluate
from river import metrics
from river import optim
from river import preprocessing
from river import linear_model
from river import drift

#### Drift Detection using DDM 

In [3]:
# Dataset
dataset = datasets.synth.Agrawal(
     classification_function=5,
     seed=42
)

# Initialise the DDM drift detector
ddm = drift.DDM(warm_start=30,
                warning_threshold=1,
                drift_threshold=2)

# Initializing model
LRmodel = pickle.load(open('../saved_models/classification/model.pkl', 'rb'))

# Initialise metric
metric = metrics.Accuracy()

print_warning = True
drifts = list()
warnings = list()

for i,data in enumerate(dataset.take(200)):
    # data[0] - features, data[1] - label
    # Data is processed one sample at a time
    y_pred = LRmodel.predict_one(data[0])
    metric = metric.update(data[1],y_pred)
    # correctly_classified data if y_pred == data[1] for checking accuracy
    ddm.update(y_pred == data[1])
    if ddm.warning_detected and print_warning:
        # The warning detector indicates after each sample if there is a warning in the data
        print(f'Warning detected at index {i}')
        warnings.append(i)
        print_warning = False
    if ddm.drift_detected:
        # The drift detector indicates after each sample if there is a drift in the data
        print(f"Change detected at index {i}")
        drifts.append(i)
        print_warning = True

print(f'{metric}\n')
print(f'Warnings: {warnings}')
print(f'Drifts: {drifts}')

Warning detected at index 34
Change detected at index 64
Warning detected at index 96
Change detected at index 159
Warning detected at index 191
Accuracy: 68.00%

Warnings: [34, 96, 191]
Drifts: [64, 159]
